# Plott sirkelbuer og avstander veg  - antatt posisjon kamera

Inpsirert av https://towardsdatascience.com/a-very-simple-demo-of-interactive-controls-on-jupyter-notebook-4429cf46aabd 

Klippet fra 
https://github.com/tirthajyoti/Interactive_Machine_Learning/blob/master/Curve_fit_widget_1.ipynb 

# Problemstilling: Vegbilder og "10 meter fram" 

Vegbildene tas fra en plattform med state-of-the-art navigasjonsplattform, med GNSS, trippteller og whatnot. Av ukjente grunner tar man data fra dette posisjoneringssystemet og ber om vegnettstilknytning 10meter **foran** bilen. Dette gir en del interessante geometriutfordringer som vi her skal utforske. 

I tillegg er dette en god problemstilling for interactive widgets. 

Parameter **Svingradius** er den reelle svingradiusen til målebilen. 

**TilSide** er avstanden fra målebil til midten av vegen, der vegnettet typisk befinner seg. 

Her viser vi hvordan geometrien blir i en venstresving. Målebilen kjører rakt fram fra venstre mot høyre. Bilen gjetter på at reell posisjon er 10m foran egentlig posisjon. Reell bevegelse avviker selvsagt når bilen følger en sving. 

![eksempel](sirkelbueplott_fig1.png) 

Hvis du har behov for den opprinnelige plasseringen (f.eks. for å finne plassering til objekter i bildet?), eller nær sagt et hvilket som helst annet analysebehov hvor det hadde vært kjekt med mer presis plassering enn størrelsesorden $\pm$ 10m, så må du regne dette ut via trigonometri og kunnskap om svingradius på stedet. 

## I stedet burde vi gjøre slik: 

![eksempel](sirkelbueplott_fig2.png) 

Bruk bilens reelle posisjon i øyeblikket, og finn nærmeste punkt på vegnettet! Da unngår vi masse kompliserte beregninger hvis og når det blir aktuelt å gjøre mere gøy med bildene  

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline

In [16]:
def finnavstand( Radius ): 
    h1 = -1*Radius + np.sqrt( 4 * Radius**2 + 400   ) / 2 
    h2 = -1*Radius - np.sqrt( 4 * Radius**2 + 400   ) / 2 
    h = max( [h1, h2])
    return h

def sirkelbue( Metode, Svingradius, SvingRetning, TilSide ): 
    # Plotter et utsnitt av sirkelbue 
    Offset = TilSide # Avstand til gul midtstripe / vegnett. 
    Radius = Svingradius
    sving = SvingRetning
    
    if Metode == 'ViaTech': 
        enkel = False
    else: 
        enkel = True 
    
    # Tegner full sirkel
    debug = False 
    
    # Setter opp figur og akser
    plt.figure(figsize=(8,5))
    ax = plt.gca()
    
    # finner theta, vinkelbue for utkjørt distanse som tilsvarer vektor [0,10], men langs sirkelbuen
    theta0 = np.arctan( [ 10 / Radius ] )[0]  
    th0 = theta0 # + 1 * np.pi / 2 # Legger på for å plotte på annet sted langs sirkelbuen
    
    
    # Setter en del parametre ut fra om det er høyre- eller venstresving
    if sving == 'Venstre': 
        thetaBasis = 3 * np.pi / 2
        fortegn = 1
    else: 
        thetaBasis = np.pi / 2
        fortegn = -1
       
    # DEFINISJON Normalpunkt = punkt på sirkelbue som er nærmest der målelinja legges
    
    # Sirkelbue som representerer kjøretøyets krumme bane (sving) til normalpunkt
    theta = np.linspace( thetaBasis, thetaBasis + fortegn * th0 , 100) 
    x = Radius * np.cos( theta )
    y = Radius * np.sin( theta ) + Radius * fortegn


    # Sirkelbue nr 2, som representerer senterlinje fram til normalpunkt 
    R2 = Radius - Offset * fortegn
    x2 = R2 * np.cos( theta )
    y2 = R2 * np.sin( theta ) + Radius * fortegn

    # Kjørt avstand langs sirkelbue 
    dR = Radius * theta0 
    dRVegnett = R2 * theta0
    
    
    # Rak pil 10m rett fram = der målelinja legges
    plt.arrow( 0, 0, 10, 0, color='b', width=0.1, length_includes_head=True )     
    # Symboliserer målelinje som 2m bred tverrstripe
    plt.plot( [10, 10], [-1, 1], '-', color='b', lw='3' )
    
    # Beregner vinkel for 10m langs vegnettets sirkelbue, og plotter denne 
    ThetaVeg = np.linspace( thetaBasis, thetaBasis + fortegn * 10 / R2, 100) 
    x3 = R2 * np.cos( ThetaVeg )
    y3 = R2 * np.sin( ThetaVeg ) + Radius * fortegn 
    # Plotter vegnettsgeometri 
    plt.plot( x3, y3, '--', c='y', lw=10)
    
    # Regner ut avstand påstått kameraposisjon - det punktet der bilen vil være målelinja
    # når bilen følger en sirkelbue 
    h = finnavstand( Radius )
    
    # Avstand til vegnett (senterlinje veg)
    # Bruker pythagoras, for denne delen av problemet er ikke symmetrisk i høyre og venstresving
    h_veg = np.sqrt((x3[-1] - 10)**2 + ( y3[-1] - 0)**2)

    
    
    
    # Markerer startpunkt og legger på tekst. 
    # plt.text( 0, -0.5, 'Målebil posisjon' ) # , ha='center')
    plt.annotate( 'Målebil posisjon', xy=(0,0), xytext=(0,-1), arrowprops=dict(facecolor='black', width=0.5 ))
    
    plt.annotate( 'Vegnett posisjon', xy=(0,Offset), xytext=(0,Offset+1), 
                 arrowprops=dict(facecolor='black', width=0.5 ))
        
                                               
    if not enkel: 
        print(  'Avstand senterlinje framskrevet 10m - målelinje', str(round( h_veg, 2)) + 'm') 
        
        
        # Målebilens bevegelse langs kurven 
        plt.plot( x, y,c='k', lw=3 )        

        # Tegner rød linje som viser differanse påstått posisjon og det punktet 
        # der kjøretøyet passerer påstått posisjon langs sirkelbue
        plt.plot( [x[-1], 10], [y[-1], 0], 'r-', lw=1)
    
        # Setter på litt flere merkelapper hvis kjøretøyet er plassert til side for 
        # vegnettet (senterlinja / gulstripa)
        if Offset > 0: 
#             print( 'Avstand påstått kameraposisjon - posisjon vegnett=', str( round( h_veg, 2)) + 'm'  ) 
#             print( 'Påstått distanse langs vegnettet =', str( round( dRVegnett, 2))+'m')
            
            # Tar kun denne med i venstresving
            if sving == 'Venstre': 
                pass
#                 plt.text( x2[-1]+0.2, y2[-1], 'Vegnett påstått posisjon, ' + str( round( dRVegnett, 2)) + 'm fra start' )
                    
        # Labels, labels
        plt.text( 10.1, 0, 'Målelinje (10 m rett fram)')
#         plt.text( x[-1]+0.2, y[-1], 'Påstått kameraposisjon tilsvarer ' + str( round( dR, 2)) + 'm langs sirkelbue')

        xtext = np.mean( [10, x[-1]]  )
        ytext = np.mean( [0, y[-1]]  )
#         plt.text( xtext+0.5, ytext, 'Avstand påstått - sirkelbue: ' + str( round( h, 2)) + 'm', color='r')

        # Rød stiplet linje viser forskjell vegnettposisjon - målelinja
        plt.plot( [x3[-1], 10], [y3[-1], 0], '--', c='r', lw=1)        
                
    else:
        # Enklere løsning, slik vi vil ha det! 
        # Tegner med hvit farge 
#         plt.plot( [x3[-1], 10], [y3[-1], 0], '--', c='w', lw=1)
        plt.plot( x3, y3, '--', c='y', lw=10)

        plt.text( 10.1, 0, 'Målelinje (10m fram)')
        
    
    
    # Full sirkel, for morro skyld og debugging
    if debug: 
        thetaFull = np.linspace( 0, 2*np.pi, 100)
        xfullsirkel = Radius * np.cos( thetaFull )
        yfullsirkel = Radius * np.sin( thetaFull ) + Radius * fortegn
        plt.plot( xfullsirkel, yfullsirkel )
        
    ax.set_aspect('equal', 'datalim')
    # plt.box( False ) # Depreceated 
    plt.gca().axis( 'Off')
    plt.show( )
#     return theta

In [17]:
y = interactive( sirkelbue, Metode=['ViaTech', 'Enkel'], 
                Svingradius=[10, 15, 20, 25, 30, 40, 50, 75, 100], 
                SvingRetning=['Venstre', 'Høyre'], 
                TilSide=[0, 0.5, 1, 1.5, 2, 3, 4, 5] )
display( y) 

interactive(children=(Dropdown(description='Metode', options=('ViaTech', 'Enkel'), value='ViaTech'), Dropdown(…